## Consultas API SDOT Pay Station

In [8]:
import os
import urllib.request
from datetime import datetime

def dl_data(url, output):
    try:        
        f = urllib.request.urlopen(url)
        print ("Downloading " + url)

        with open(os.path.basename(output), "wb") as local_file:
            local_file.write(f.read())

    except URLError:
        print ("Error", url)

In [9]:
# Url 1000 primeros ids:
url0 = 'http://gisrevprxy.seattle.gov/arcgis/rest/services/SDOT_EXT/DSG_datasharing/MapServer/54/query?where=1%3D1&text=&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=*&returnGeometry=false&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&returnDistinctValues=false&resultOffset=&resultRecordCount=&queryByDistance=&returnExtentsOnly=false&datumTransformation=&parameterValues=&rangeValues=&f=pjson'

In [10]:
output0 = './data/paystations_ids_1_1000.json'
dl_data(url0, output0)

In [26]:
# Url 800 siguientes ids (Result Offset = 1000, Result Record Count = 800):
url1 = 'http://gisrevprxy.seattle.gov/arcgis/rest/services/SDOT_EXT/DSG_datasharing/MapServer/54/query?where=1%3D1&text=&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=*&returnGeometry=false&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&returnDistinctValues=false&resultOffset=1000&resultRecordCount=800&queryByDistance=&returnExtentsOnly=false&datumTransformation=&parameterValues=&rangeValues=&f=pjson'

In [27]:
output1 = '.data/paystations_ids_1001_1800.json'
dl_data(url1, output1)

## Consolidación de respuesta en fichero csv

In [5]:
import pandas as pd
import os

static_data_0 = pd.read_json('./data/paystations_ids_1_1000.json')
static_data_1 = pd.read_json('./data/paystations_ids_1001_1800.json')
static_data = pd.concat([static_data_0, static_data_1], axis=0, sort=True)

In [6]:
static_data = static_data.attributes.to_frame()
static_data.attributes.iloc[0].keys()

static_data['element_key'] = static_data.attributes.map(lambda dic: dic['ELMNTKEY'])
static_data['latitude'] = static_data.attributes.map(lambda dic: dic['SHAPE_LAT'])
static_data['longitude'] = static_data.attributes.map(lambda dic: dic['SHAPE_LNG'])
static_data.drop('attributes', axis=1, inplace=True)

In [7]:
sd_group_ek = static_data.groupby('element_key')['latitude', 'longitude'].mean()
sd_group_ek = sd_group_ek.reset_index(level=['element_key'])
sd_group_ek.to_csv('./data/Coord_EK.csv', index=False)

In [8]:
coord = pd.read_csv(os.path.join('./data/Coord_EK.csv'))
coord.head()

,element_key,latitude,longitude
0,1001,47.602862,-122.334703
1,1002,47.602997,-122.334538
2,1005,47.603602,-122.335382
3,1006,47.603725,-122.335171
4,1009,47.605010,-122.336669
